In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random 

gas_constant = 0.83144598
Boltzmann = 0.001987204

In [ ]:
class molecular_dynamics:


    def __init__(self,box_length,position,time_step,temperature,mass):
        self.box_length = box_length
        self.time_step = time_step
        self.position = position 
        self.position = np.array(self.position)
        self.num_particles = len(self.position)
        self.temperature = temperature #Kelvin
        self.mass = mass # AMU
        self.cutoff = box_length / 2
        self.velocity = self.__init_velocity__()
        self.acceleration = self.__calc_acceleration_reflection__(self.mass, False)
    
        
    def MD_run(self, steps): 
        

        displacement_array = []
        displacement_average_array = []
        potential_energy_array = []
        kinetic_energy_array = []
        temperature_array = []
        displacement_array = []
        velocities_array = []


        with open("reflection.xyz", 'w') as file:
            # Write initial step
            file.write("{}\n".format(len(self.position)))
            file.write("Step {}\n".format(0))
            for pos in self.position:
                file.write("{} {} {} {}\n".format("Ar", pos[0], pos[1], pos[2])) 

            for i in range(steps):
                
                self.__reflection_boundary_condition__()
                acceleration_update, pot_energy = self.__calc_acceleration_reflection__(self.mass, True)


                kinetic_energy, temperature_instant = self.__kinetic_energy_temperature_cal__()
                self.velocity = self.__velocity_update__(self.velocity, self.acceleration, acceleration_update)
                
                scaling = np.sqrt(self.temperature/temperature_instant)
                self.velocity *= scaling

                velocities_array.append(self.velocity)
                kinetic_energy_array.append(kinetic_energy)
                temperature_array.append(temperature_instant)

                potential_energy_array.append(pot_energy)
                
                
                self.acceleration = acceleration_update


            
                file.write("{}\n".format(len(self.position)))
                file.write("Step {}\n".format(i + 1))  
                for pos in self.position:
                    file.write("{} {} {} {}\n".format("Ar", pos[0], pos[1], pos[2])) 


                distance_init_current = LJC_13_initial - self.position 
                distance_init_current_array = np.array(distance_init_current)
                

                
                for i in range(len(distance_init_current_array)):
                    displacement = np.linalg.norm(distance_init_current_array[i])**2
                    displacement_array.append(displacement)

                displacement_average = np.mean(displacement_array)
                displacement_average_array.append(displacement_average)
               

        return potential_energy_array, kinetic_energy_array, temperature_array, displacement_average_array, velocities_array, 


    def __reflection_boundary_condition__(self):
        speed, direction = self.__speed_direction__(self.velocity)
        for k in range(self.num_particles):
            for l in range(len(self.position[k])):
                self.position[k][l] = self.__position_update__(self.position[k][l], speed[k], direction[k][l], self.acceleration[k][l])
                
                if self.position[k][l] > self.box_length:
                    self.position[k][l] = (self.box_length * 2) - self.box_length
                    direction[k][l] = -direction[k][l]
                    self.velocity[k][l] = speed[k]*direction[k][l]
                
                elif self.position[k][l] < -self.box_length:
                    self.position[k][l] = -(self.box_length * 2) + self.box_length
                    direction[k][l] = - direction[k][l]
                    self.velocity[k][l] = speed[k]*direction[k][l]
             
    


    def __calc_acceleration_reflection__(self, mass, pot_energy_option):
        
        acceleration = np.zeros((self.num_particles, 3)) 
        pot_energy = 0 
        
        for i in range(self.num_particles): 
            for j in range(i + 1, self.num_particles):
                
                distance_vector_ij = self.position[i] - self.position[j] 
                distance_ij = np.linalg.norm(distance_vector_ij) 
                derivative_potential_ij = self.__derivative__lennard_jones_potential__(distance_ij, 1, 1) 
                force_components = -derivative_potential_ij * distance_vector_ij / distance_ij 
                acceleration[i] += force_components / mass
                acceleration[j] -= force_components / mass
                
                if pot_energy_option:
                    pot_energy += self.__lennard_jones_potential__(distance_ij, 1, 1)
        
        if pot_energy_option:
            return acceleration, pot_energy
        else:
            return acceleration

        
    def __position_update__(self, position, speed, direction, acceleration):
        position += ((speed * direction) * self.time_step) + ((0.5*acceleration) * (self.time_step**2))
        return position
    

    def __init_velocity__(self):
        standard_deviation = np.sqrt((self.temperature * Boltzmann) / self.mass)
        self.velocity = np.random.normal(loc=0, scale=standard_deviation, size=(self.num_particles, 3))
        return self.velocity

    def __speed_direction__(self,velocity):
        direction = []
        speed = []
        for v in velocity:
            speed.append(np.linalg.norm(v))
            direction.append(v / np.linalg.norm(v))
        return speed, direction


    def __velocity_update__(self, velocity, acceleration_0, acceleration_1): 
        velocity += + 0.5*(acceleration_0 + acceleration_1)*self.time_step
        return velocity
    
    def __velocity_rescale__(self):
        kinetic_energy, temp_instant = self.__kinetic_energy_temperature_cal__()
        self.velocity = self.velocity * (np.sqrt(self.temperature/temp_instant))
        scaled_kinetic_energy, scaled_temp = self.__kinetic_energy_temperature_cal__()
        return scaled_kinetic_energy, scaled_temp 
        


    def __kinetic_energy_temperature_cal__(self):
        kinetic_energy = 0
        temperature = 0
        for v1 in self.velocity:
            kinetic_energy += 0.5* self.mass * (np.linalg.norm(v1) ** 2)
        temperature = 2 * kinetic_energy/(3* self.num_particles * Boltzmann) 
        return kinetic_energy, temperature


    def __lennard_jones_potential__(self, r, sigma, epsilon):
        return 4 * epsilon * np.power(sigma / r, 12) - 4 * epsilon * np.power(sigma / r, 6) 
         
         
    def __derivative__lennard_jones_potential__(self, r, sigma, epsilon):
        return -((24 * epsilon) * ((2 * sigma**12 / r**13) - (sigma**6 / r**7)))

        


 